Load Pandas

In [31]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

Load data

In [32]:
data = pd.read_csv("jester-data-1.csv", header = None)

In [93]:
data_df = data.drop(data.columns[[0]], axis=1)
data_df2 = data_df.copy()
i = 1
p = 1
x = 0
changes_user = []
changes_item = []
n_features = 2
user_ratings = data_df.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))


while x < data_df.shape[0] * 0.1:
    i = np.random.randint(1,data_df2.shape[0])
    p = np.random.randint(1,data_df2.shape[1])
    #print(i, ",", p)
    if data_df2.iloc[i][p] != 99:
        changes_user.append(i)
        changes_item.append(p)
        #print(i, ",", p)
        #print("---")
        data_df2.iloc[i][p] = 99
        #print(data_df2.iloc[i][p])
        #print(data_df.iloc[i][p])
        x += 1
    
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [94]:
latent_item_features

array([[0.545, 0.735],
       [0.789, 0.51 ],
       [0.58 , 0.664],
       [0.972, 0.609],
       [0.316, 0.349],
       [0.706, 0.832],
       [0.066, 0.54 ],
       [0.787, 0.557],
       [0.008, 0.941],
       [0.555, 0.931],
       [0.467, 0.772],
       [0.26 , 0.903],
       [0.851, 0.08 ],
       [0.213, 0.01 ],
       [0.407, 0.707],
       [0.31 , 0.898],
       [0.814, 0.761],
       [0.946, 0.126],
       [0.086, 0.9  ],
       [0.598, 0.969],
       [0.253, 0.951],
       [0.93 , 0.536],
       [0.796, 0.831],
       [0.794, 0.257],
       [0.882, 0.688],
       [0.725, 0.985],
       [0.277, 0.646],
       [0.306, 0.071],
       [0.364, 0.612],
       [0.189, 0.556],
       [0.046, 0.716],
       [0.859, 0.902],
       [0.787, 0.964],
       [0.923, 0.18 ],
       [0.67 , 0.49 ],
       [0.331, 0.297],
       [0.083, 0.028],
       [0.688, 0.701],
       [0.278, 0.595],
       [0.176, 0.251],
       [0.295, 0.685],
       [0.728, 0.875],
       [0.588, 0.075],
       [0.2

In [95]:
latent_user_preferences

array([[0.832, 0.036],
       [0.394, 0.455],
       [0.886, 0.88 ],
       ...,
       [0.608, 0.101],
       [0.466, 0.019],
       [0.009, 0.009]])

In [96]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 30):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(not np.isnan(rating) and rating != 99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%10000 == 0 ):
            print(mse)

In [ ]:
for _ in range(0,5): 
    sgd()

18.108971247208473
17.14823488120747


In [127]:
predictions = []
real = []

for i in range(0,len(changes_user)):
    predictions.append(predict_rating(changes_user[i],changes_item[i]))
    real.append(data_df.iloc[changes_user[i]][changes_item[i]])
#print(real)
#print(predictions)
  

[-5.53, -0.39, -3.64, 5.68, 2.67, -9.51, 2.77, 9.03, 9.08, 8.01, -2.67, -4.42, 8.01, 9.03, 0.24, 4.32, 4.17, 4.13, 1.99, 2.67, 9.22, 4.27, 2.96, 9.03, 2.18, -3.01, 1.17, -9.51, 5.92, 4.9, 4.27, -8.83, -1.99, 8.83, 4.95, -5.78, 0.92, -1.89, -0.49, 1.6, 2.18, -3.64, 0.73, 2.14, 1.12, 6.31, 1.31, 5.39, -6.7, -4.03, 6.75, -8.64, 8.88, 2.09, 7.38, 3.16, 6.99, 4.95, 3.45, -4.81, -0.29, 4.76, -8.83, 3.64, 5.24, 2.48, -3.06, 3.01, 0.63, 1.07, 1.94, 0.73, -4.32, 2.57, -8.5, 8.98, -9.08, 3.01, 2.62, 9.32, 5.29, 2.52, 0.78, 9.17, 1.07, -9.51, -1.99, -1.7, 1.84, -0.92, 2.77, -3.59, -3.93, 3.25, -8.01, 8.59, 5.1, 7.09, 6.75, -7.23, 0.24, -8.45, 3.16, 0.15, -7.57, -0.44, 1.17, 1.94, -2.72, 4.47, 7.96, 7.23, 4.71, 0.97, -5.63, 2.67, 1.99, 1.21, -3.83, 3.83, 3.16, 6.65, 5.49, -3.54, -0.53, 3.64, 4.56, 0.0, -6.46, 8.64, 1.99, 2.67, -5.15, 0.87, 5.73, 2.18, 3.11, 6.07, -0.34, -3.25, -1.94, 8.3, 8.88, -4.71, -1.75, 8.45, 5.87, 2.82, -1.12, -0.97, 0.29, -1.12, -0.1, 7.48, 0.29, -6.99, -7.43, 3.01, -3.45, 

In [126]:
error = 0
for e in range(0,len(changes_user)):
    error = error + (real[e]-predictions[e])**2
error = error/len(changes_user)
print(error)

24.837047192624954
